### reference1: https://www.kaggle.com/chenyc15/mean-average-precision-metric and edited herein
### reference2: https://www.kaggle.com/cchadha/mean-average-precision-iou-on-cnn-oof-preds

### RSNA Competition은 IoU 임계치에 따른 mAP를 구해서 submission.csv 평가한다. IoU는 두 bounding box가 일치하는 정도를 나타내며 IoU 각 임계치(0.4 to 0.75까지 0.05 단위, (0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75))에서 mAP를 구한다.

### 1.  Bbox, image, model과 AP, mAP 관계
#### - bounding box의 임계치 별 IoU를 구해서 n개의 Precision 구함
#### - image 1개는 임계치 별 IoU의 Precision 평균인 AP 1개 출력
#### - 모든 사진의 AP를 평균 낸 값으로 모델의 mAP를 계산

### 2. AP계산방법
#### - ground truth와 prediction box 둘 다 값이 없으면 그냥 패스
#### -  ground truth와 prediction box 둘 중 하나가 0이면 그 image의 FP가 0이므로 AP도 0이므로 return 0
#### - ground truth와 prediction box 모두 양수이면 prediction box들을 score에 대해 내림차순으로 정렬한 후 차례대로 truth box랑 매칭해서 tp,tn,fp 구함.
#### (iou> threshold) ->  TP(정탐)
#### (iou< threshold) ->  FP(미탐)
#### (예측한 bbox - 찾은 bbox) -> FN(오탐)
#### 이후, 각 threshold에서 (tp/tp+fp+tn) 구한 후 thresholds 개수로 나눠서 AP 구한다.

##### iou()는 두개의 바운딩 박스를 비교해서 일치정도를 계산하는 함수이다.
##### map_iou()는 RSNA Validation 방식을 코드로 구현한 함수이다.

In [ ]:
# helper function to calculate IoU
def iou(box1, box2):
    x11, y11, w1, h1 = box1
    x21, y21, w2, h2 = box2
    assert w1 * h1 > 0
    assert w2 * h2 > 0
    x12, y12 = x11 + w1, y11 + h1
    x22, y22 = x21 + w2, y21 + h2

    area1, area2 = w1 * h1, w2 * h2
    xi1, yi1, xi2, yi2 = max([x11, x21]), max([y11, y21]), min([x12, x22]), min([y12, y22])
    
    if xi2 <= xi1 or yi2 <= yi1:
        return 0
    else:
        intersect = (xi2-xi1) * (yi2-yi1)
        union = area1 + area2 - intersect
        return intersect / union

In [ ]:
# 한 사진에 대해서 mAP 값을 구하는 함수이다.
def map_iou(boxes_true, boxes_pred, scores, thresholds = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75]):
    """
    Mean average precision at differnet intersection over union (IoU) threshold
    
    input:
        boxes_true: Mx4 numpy array of ground true bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
                    ground truth BBox 배열 or 배열들 (x, y, w, h) 
        boxes_pred: Nx4 numpy array of predicted bounding boxes of one image. 
                    bbox format: (x1, y1, w, h)
                    예측한 bbox 배열 or 배열들 (x, y, w, h)
        scores:     length N numpy array of scores associated with predicted bboxes
        thresholds: IoU shresholds to evaluate mean average precision on
    output: 
        map: mean average precision of the image
    """
    
    # According to the introduction, images with no ground truth bboxes will not be 
    # included in the map score unless there is a false positive detection (?)
        
    # return None if both are empty, don't count the image in final evaluation (?)
    """
    모두 0이면 dont count
    하나는 0이고 하나는 0이 아니면, TP가 0이므로 return 0을 한다.
    둘 다 하나 이상의 값이 있으면 TP, FN, FP 계산이 필요하다.
    """
    if len(boxes_true) == 0 and len(boxes_pred) == 0:     
        return None
    elif len(boxes_true) == 0 and len(boxes_pred) > 0:  
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) == 0:
        return 0
    elif len(boxes_true) > 0 and len(boxes_pred) > 0:
        assert boxes_true.shape[1] == 4 or boxes_pred.shape[1] == 4, "boxes should be 2D arrays with shape[1]=4"
        if len(boxes_pred):
            assert len(scores) == len(boxes_pred), "boxes_pred and scores should be same length"
            # sort boxes_pred by scores in decreasing order, bbox를 scores에 따른 내림차순 정렬 
            boxes_pred = boxes_pred[np.argsort(scores)[::-1], :] 

        map_total = 0

        # loop over thresholds
        for t in thresholds:
            matched_bt = set()
            tp, fn = 0, 0
            for i, bt in enumerate(boxes_true):
                matched = False
                for j, bp in enumerate(boxes_pred):
                    miou = iou(bt, bp)
                    if miou >= t and not matched and j not in matched_bt:
                        matched = True # IoU가 t임계치를 넘었으므로 정탐
                        tp += 1 # bt is matched for the first time, count as TP
                        matched_bt.add(j)
                if not matched: # 모든 bbox t임계치를 넘지 못했으므로 미탐
                    fn += 1 # bt has no match, count as FN
                    
            # 예측한 bbox 중에 정탐한 bbox를 제외하면 나머지 bbox는 오탐으로 간주
            fp = len(boxes_pred) - len(matched_bt) # FP is the bp that not matched to any bt
            m = tp / (tp + fn + fp)
            map_total += m # 모든 임계치에 대해서 m 값을 계산
    
    return map_total / len(thresholds) #kaggle validation function